<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1lstm_d7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # تبدیل y_true به float32
        y_true = K.cast(y_true, dtype=tf.float32)

        # Clip predictions to prevent log(0) errors
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        # Compute cross-entropy loss
        cross_entropy = -y_true * K.log(y_pred)

        # Apply focal loss formula
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        return K.sum(loss, axis=-1)

    return focal_loss_fixed

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ بارگذاری دیتاست شخصی (لطفاً نام فایل را تغییر بده!)
file_path = "/content/d7.csv"  # نام فایل خود را اینجا بگذار
df = pd.read_csv(file_path)

# 2️⃣ بررسی ساختار داده‌ها
print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

# 3️⃣ مشخص کردن ویژگی‌های ورودی و خروجی
# نام ستون‌ها را بر اساس دیتاست خود تغییر بده!
feature_columns = ["A","B","C","D","E"]  # نام ویژگی‌ها
target_column = "F"  # نام ستون هدف (برچسب‌ها)

# تبدیل دیتافریم به آرایه NumPy
X_data = df[feature_columns].values
y_data = df[target_column].values

# مشخص کردن تعداد ویژگی‌ها
num_features = X_data.shape[1]

# 4️⃣ تنظیم داده‌ها برای مدل LSTM
time_steps = 30  # تعداد مراحل زمانی (مثلاً ۱۰ ورود اخیر هر کاربر)

# ایجاد دنباله‌های زمانی برای LSTM
X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])  # داده‌های ۱۰ زمان اخیر
    y_sequences.append(y_data[i + time_steps])  # مقدار هدف بعد از ۱۰ مرحله

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 5️⃣ تقسیم داده‌ها به مجموعه آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 6️⃣ مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

# 7️⃣ ایجاد مدل LSTM
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 8️⃣ کامپایل مدل
model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

# 9️⃣ آموزش مدل روی دیتای شخصی
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 🔟 ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

# ۱۱️⃣ پیش‌بینی روی یک نمونه جدید
test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")

🔍 بررسی ۵ سطر اول دیتاست:
   A  B  C  D  E  F
0  7  6  1  2  1  0
1  2  2  0  4  6  0
2  7  7  3  7  6  0
3  5  0  2  1  2  0
4  2  4  0  1  8  0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 46s 32ms/step - accuracy: 0.6020 - loss: 5.1490e-04 - val_accuracy: 0.5904 - val_loss: 6.2763e-09
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 38s 31ms/step - accuracy: 0.6024 - loss: 5.4558e-08 - val_accuracy: 0.5904 - val_loss: 1.3334e-09
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.5991 - loss: 2.2323e-08 - val_accuracy: 0.5904 - val_loss: 4.2681e-10
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.5991 - loss: 8.3480e-09 - val_accuracy: 0.5904 - val_loss: 1.7883e-10
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - accuracy: 0.5993 - loss: 3.6585e-09 - val_accuracy: 0.5904 - val_loss: 7.6238e-11
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.6016 - loss: 2.2688e-09 - val_accuracy: 0.5904 - val_loss: 3.1819e-11
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 33ms/step - accuracy: 0.6005 - loss: 1.9103e-09 - val_accuracy: 0.5904 - val_loss: 1.5539e-11
Epoch 8/10
1250/1250